In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import  DateFormatter
# %matplotlib inline
import fitsio
from fitsio import FITS, FITSHDR
import datetime
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
import time

In [2]:
from some_func import cart2spheric, spheric2cart, spheric_coordinates_rotate, hor2pix_s1c, bright_relay_s1c

In [3]:
# Координаты расположения S1C
lat_s1c=56.1501667 # ШИРОТА
lon_s1c=46.1050833 # ДОЛГОТА
h_s1c=183.0
S1C_site=EarthLocation(lat=lat_s1c*u.deg, lon=lon_s1c*u.deg, height=h_s1c*u.m)

# Ориентация S1C - 24 августа
az0_s1c = 5.25221062496641;
alt0_s1c = 1.5541049224175232;
a_s1c = (-0.2348695544502367, 0.0004395865611329251, 0.001172237836255852)
b_s1c = (0.10820004418209775, -0.0011720136387002968, 0.000437125282422439)
c_s1c = ( 146.55099832248433, 279.114199528675, -748.5277665569147)
d_s1c = ( 145.4040578902056, 748.3731317338136, 280.7000564862628)

In [4]:
dark_filename='/mnt/Work_disk/Owncloud/0000_Two_points/DATA/01_S1C/140824/dark/s1c_aug24_dark_median.fit'
dark=fitsio.read(dark_filename)
dark.astype('d')
BS6300_catalog_filename='Haritonov_6300BS_catalog.npz'

In [5]:
npzfile=np.load(BS6300_catalog_filename)
CAT_FIELDS=npzfile['CAT_FIELDS']
CAT_6300BS=npzfile['CAT_6300BS']
CAT_SPTYPE=npzfile['CAT_SPTYPE']
print(CAT_FIELDS)

['HARITONOV_ID' 'BS_ID' 'RA (icrs, deg)' 'DEC (icrs, deg)' 'MAG_V'
 '6300 Flux (erg/(sm^2 * s *  sm))']


In [6]:
img_num_start=1
# img_num_end=1
img_num_end=515
img_num_list=[]
R_median_list=[]
R_median_sum=0.
time_axe=[]
ind=0
for img_num in range(img_num_start,img_num_end+1):
#     pic_save_name='/home/ashindin/temp/' + "{0:0>3}".format(img_num) + '.png'
    ind+=1
    pic_save_name='/home/ashindin/temp/s1c_aug24/' + "{0:0>4}".format(ind) + '.jpg'
    fig=plt.figure(figsize=(9,4.5))
    ax = plt.subplot(121)
    ax.grid(True)
    ax.set_xlim((100, 5000))
    ax.set_ylim((0, 5))
    plt.ylabel('Calibration coef. [R/ADCu]')
    plt.xlabel('Star brightness per pixel [R]')
    ax2=plt.subplot(122)
    ax2.set_xlim((1,288))
    ax2.set_ylim((288,1))
        
    fit_filename='/mnt/Work_disk/Owncloud/0000_Two_points/DATA/01_S1C/140824/aug24red15bin2_0' + "{0:0>3}".format(img_num) + '.fit'

    fitfile=fitsio.FITS(fit_filename)
    img=fitfile[0].read()
    img=img.astype('d')-dark
    plt.axes(ax2)
    plt.pcolormesh(img, cmap="gray", vmin=np.median(img)-100, vmax=np.median(img)+100)
    header=fitfile[0].read_header()
    fitfile.close()
    date_obs_str=header['COMMENT'][7:31]
    if date_obs_str[16]==' ':
        date_obs_str=date_obs_str[0:16] + '0' + date_obs_str[17:24]
    if date_obs_str[17]==' ':
        date_obs_str=date_obs_str[0:17] + '0' + date_obs_str[18:24]

    date_obs=datetime.datetime.strptime(date_obs_str,"%b %d %H:%M:%S.%f %Y") - datetime.timedelta(seconds=7.5) - datetime.timedelta(hours=4)
    plt.title('fr'+"{0:0>4}".format(img_num) + ' ' + str(date_obs)[0:23])
    time_axe.append(date_obs)
    
    print("{0:0>3}".format(img_num) + " "+ str(date_obs))

    BS_coord=SkyCoord(CAT_6300BS[:,2], CAT_6300BS[:,3], frame='icrs', unit='deg');
    
    altaz=BS_coord.transform_to(AltAz(obstime=date_obs, location=S1C_site,temperature=15*u.deg_C,pressure=1013*u.hPa,
                                       relative_humidity=0.5,obswl=630.0*u.nm))
    BS_AzAlt=np.zeros((np.size(CAT_6300BS,0),2))
    BS_AzAlt[:,0]=altaz.az.degree
    BS_AzAlt[:,1]=altaz.alt.degree
#     print(BS_AzAlt[28,0],BS_AzAlt[28,1])

    BS_xy=np.zeros((np.size(CAT_6300BS,0),2))
    BS_xy[:,0], BS_xy[:,1] = hor2pix_s1c (BS_AzAlt[:,0]*np.pi/180,BS_AzAlt[:,1]*np.pi/180, az0_s1c, alt0_s1c, c_s1c, d_s1c)

    # Catalog filtration
    filt_mask=np.zeros(np.size(CAT_6300BS,0),dtype=bool)
    for i in range(0,np.size(CAT_6300BS,0)):
        if BS_AzAlt[i,1]>=85.0:
            if BS_xy[i,0]>=1 and BS_xy[i,0]<=288:
                if BS_xy[i,1]>=1 and BS_xy[i,1]<=288:
                    filt_mask[i]=True
    BS_AzAlt_filt=BS_AzAlt[filt_mask,:]
    BS_xy_filt=BS_xy[filt_mask,:]
    CAT_6300BS_filt=CAT_6300BS[filt_mask,:]
    CAT_SPTYPE_filt=CAT_SPTYPE[filt_mask]
    if len(CAT_6300BS_filt)!=0:
        img_red_st_int=np.zeros(np.size(BS_xy_filt,0))
        img_red_st_num=np.zeros(np.size(BS_xy_filt,0))
        BS_adc_filt=np.zeros(np.size(BS_xy_filt,0))
        area_rad=4
        for j in range(np.size(BS_xy_filt,0)):    
            st_x=BS_xy_filt[j,0]
            st_y=BS_xy_filt[j,1]
            area=img[int(st_y)-area_rad:int(st_y)+area_rad+1, int(st_x)-area_rad:int(st_x)+area_rad+1]

            sum_temp=0.0
            num=0
            med=np.median(area)
            for i in range(len(area.flat)):
                if area.flat[i]>=1.3*med:
                    num+=1
                    sum_temp+=area.flat[i]-med
            img_red_st_int[j]=sum_temp
            img_red_st_num[j]=num
            if num>0:
                BS_adc_filt[j]=sum_temp/num

        BS_relay=np.zeros(np.size(BS_xy_filt,0))
        R_adc_coef=np.zeros(np.size(BS_xy_filt,0))
        for i in range(np.size(BS_xy_filt,0)):
             if img_red_st_num[i]>3:
                    BS_relay[i]=bright_relay_s1c(CAT_6300BS_filt[i,5],img_red_st_num[i])
                    if BS_adc_filt[i]>0 and BS_relay[i]>100:
                        R_adc_coef[i]=BS_relay[i]/BS_adc_filt[i]

    #     print(R_adc_coef[R_adc_coef.nonzero()])
        R_median=np.median(R_adc_coef[R_adc_coef.nonzero()])
        R_median_sum+=R_median

        print(R_median)
        R_median_list.append(R_median)
        img_num_list.append(img_num)
        
        plt.axes(ax)
        plt.plot([0, 5000], [R_median, R_median], c='r', lw=2)
        plt.scatter(BS_relay,R_adc_coef,s=np.pi*(img_red_st_num)**2)
        plt.ylabel('Calibration coef. [R/ADCu]')
        plt.xlabel('Star brightness per pixel [R]')

        plt.title(R_median)

        plt.axes(ax2)
        plt.plot(BS_xy_filt[:,0],BS_xy_filt[:,1],color="r",marker=".", lw=0.,mec="r", mfc="r")
        
#         for ii in range(len(CAT_6300BS_filt)):
#             ax2.text(BS_xy_filt[ii,0]-10, BS_xy_filt[ii,1]-10, str(int(CAT_6300BS_filt[ii,0])), ha="center", va="center", size=12)
        
        
        # plt.show()
    plt.savefig(pic_save_name)
    plt.close()
print(' ')
print(R_median_sum/(img_num_end-img_num_start+1))

001 2014-08-24 17:43:07.970000
2.25023516328
002 2014-08-24 17:43:22.970000
2.3285845708
003 2014-08-24 17:43:37.970000
2.48309936953
004 2014-08-24 17:43:52.970000
2.41997362939
005 2014-08-24 17:44:07.985000
2.27258654505
006 2014-08-24 17:44:22.985000
2.32484854065
007 2014-08-24 17:44:37.985000
2.474853001
008 2014-08-24 17:44:52.985000
2.39047527977
009 2014-08-24 17:45:07.985000
2.17880509377
010 2014-08-24 17:45:23.001000
2.3128913521
011 2014-08-24 17:45:38.001000
2.5009565421
012 2014-08-24 17:45:53.001000
2.39508501171
013 2014-08-24 17:46:08.001000
2.2273677009
014 2014-08-24 17:46:23.016000
2.34892613271
015 2014-08-24 17:46:38.016000
2.47979423632
016 2014-08-24 17:46:53.016000
2.39728637662
017 2014-08-24 17:47:08.016000
2.20514675157
018 2014-08-24 17:47:23.016000
2.28573094186
019 2014-08-24 17:47:38.032000
2.50552120822
020 2014-08-24 17:47:53.032000
2.40613245181
021 2014-08-24 17:48:08.032000
2.20309787799
022 2014-08-24 17:48:23.032000
2.26057165692
023 2014-08-24 1

In [9]:
dates = matplotlib.dates.date2num(time_axe)
dates2=dates[np.array(img_num_list)-1]

In [10]:
fig=plt.figure(figsize=(9,8))
ax = plt.subplot(211)
# plt.scatter(img_num_list,R_median_list)
plt.plot_date(dates2,R_median_list)
#ax.set_ylim((0,5))
ax.grid(True)
# plt.gcf().autofmt_xdate()
ax.xaxis.set_major_formatter( DateFormatter('%H:%M') )
plt.ylabel('Calibration coef. [R/ADCu]')

plt.title('S1C (24 aug 2014) calibration curve')

ax2=plt.subplot(212)
plt.plot_date(dates2,R_median_list)
ax2.set_ylim((0,5))
ax2.grid(True)
# plt.gcf().autofmt_xdate()
ax2.xaxis.set_major_formatter( DateFormatter('%H:%M') )
plt.ylabel('Calibration coef. [R/ADCu]')

# plt.show()
plt.savefig('/home/ashindin/temp/s1c_aug24.png')